In [1]:
#Importeer standard libraries
import pandas as pd
import numpy as np
from datetime import datetime
from ipywidgets import widgets
import matplotlib.pyplot as plt
#Importeer plotly libraries
import plotly
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

In [2]:
#Bestand kiezen, importeren, dateTimeIndex en van comma naar punt
#Importeer libraries 
import os
import tkinter as tk
from tkinter import filedialog
from ipywidgets import Output
from IPython.display import clear_output

#Print tekst voor de waarden die gebruikt moeten worden
print(
    '''
for Nibe Uplink: 
'buitentemp. BT1 [°C]',
'huid. compr.frequentie  [Hz]',
'temp. bron in EB100-EP14-BT10 [°C]',
'temp. bron uit EB100-EP14-BT11 [°C]',
'aanv.verwarming BT2 [°C]'
 '''
 )

#Defineer knop
button5 = widgets.Button(description='Choose file')

#Toon knop
display(button5)

#Defineer output
out2 = Output()

#Defineer dataframe
df = pd.DataFrame()

#Knop inhoud
#Kies bestand dialoog
#Vervang komma's door punten waarmee Pyhton rekent
#Zet tijd naar stappen van 2 mintuen en interpoleerd de gegevens als die afwezig zijn
def on_button5_clicked(b):
    global df, tk
    root = tk.Tk()
    root.lift()
    root.wm_attributes("-topmost", 1)
    root.focus_force()
    filepath = filedialog.askopenfilename(parent=root)
    filename_w_ext = os.path.basename(filepath)
    filename, file_extension = os.path.splitext(filename_w_ext)
    root.withdraw()
    try:     
        df = pd.read_csv(filepath, engine='python', delimiter=";", index_col=0, header=0, skiprows=None, encoding='utf-16le')
        df.set_index(pd.DatetimeIndex(df.index.values),inplace=True)
        df = df.replace(',','.', regex=True)
        df = df.replace('SERR', np.nan).dropna()
        df = df.apply(pd.to_numeric).round(3)
        df = df.resample('2T').mean().interpolate()
        df = df.replace('SERR', np.nan).dropna()
        with out2:
            clear_output()
            print('file loaded succesfully')
            print(filename + file_extension)
    except Exception as e: #foutmelding geven als iets mis gaat
        with out2:
            clear_output()
            print(e)
    
#Activeer definitie die bij de knop hoort    
button5.on_click(on_button5_clicked)


for Nibe Uplink: 
'buitentemp. BT1 [°C]',
'huid. compr.frequentie  [Hz]',
'temp. bron in EB100-EP14-BT10 [°C]',
'temp. bron uit EB100-EP14-BT11 [°C]',
'aanv.verwarming BT2 [°C]'
 


Button(description='Choose file', style=ButtonStyle())

In [3]:
out2

Output()

In [4]:
#COP Tabel invoeren
df_COP2 = pd.DataFrame({'COP' : [1.75,2.21,2.63,3.04,3.47,3.91,4.41,4.94,5.66,6.53,7.52,8.43,9.35,10.28]},
                       index=[-70.00,-65.00,-60.00,-55.00,-50.00,-45.00,-40.00,-35.00,-30.00,-25.00,-20.00,-15.00,-10.00,-5.00])

#Afronden op 5 aantallen
def myround(x, base=5):
    return int(base * round(float(x)/base))

#definitie COP vinden
def createCOP2(BI, CCF, df_COP2):
    COP = []
    for indexName,CCFthing in zip(BI, CCF): 
        if not -70 <= myround(indexName) <= -5 or CCFthing==0:
            COP.append(None) 
        else:
            COP.append(df_COP2['COP'][myround(indexName)])
    return COP

#COP vinden start
def MakeCOP2():
    Delta_T = []
    Delta_T = df['temp. bron in EB100-EP14-BT10 [°C]'] - df['aanv.verwarming BT2 [°C]']
    df['COP'] = createCOP2(Delta_T, df['huid. compr.frequentie  [Hz]'], df_COP2)

In [5]:
#Defineer invulvakken voor variabele
P = widgets.IntText(value=6, description='Power:')
F = int(90) #Was eerste iets wat geweizigd kon worden is nu een vaste waarde
A = widgets.FloatText(value=20, description='Area panels:')
ui1 = widgets.VBox([P, A], layout={'border': '0px solid black'})

In [6]:
ui1

In [7]:
df_fr = pd.DataFrame()

#Vriesminuten berekenen
def Freezeminutes(BRT, BT):
    y = 0
    Freeze = []
    for x, z in zip (BRT, BT):
        y = y + (0 - ((x + z) / 2))
        if y < 0:
            y=0
        Freeze.append(y)
    return Freeze 

#Defineer vriesminuten maken
def MakeFreezeminutes():
    df['Freezeminutes'] = Freezeminutes((df['temp. bron in EB100-EP14-BT10 [°C]'] + df['temp. bron uit EB100-EP14-BT11 [°C]'])/2, df['buitentemp. BT1 [°C]'])

In [8]:
out3 = Output()
#Delta T berekenen
def MakeDT():
    df['Delta_T'] = df[('buitentemp. BT1 [°C]')] - ((df[('temp. bron in EB100-EP14-BT10 [°C]')] + df[('temp. bron uit EB100-EP14-BT11 [°C]')])/2)

#Actueel vermogen berekenen
def MakePa():
    df['Actueel vermogen'] = (df[('huid. compr.frequentie  [Hz]')] / F) * P.value
    df['Actueel vermogen'] = df['Actueel vermogen'].apply(pd.to_numeric).round(3)
    #df['Actueel vermogen'] = df['Actueel vermogen'].round(3)

#U-waarde berekenen
def MakeU():
    df['U_waarde'] = df['Actueel vermogen'] * 1000 * (1-(1 / df['COP'])) / df['Delta_T'] / A.value
    df["U_waarde"] = pd.to_numeric(df["U_waarde"])
    df['U_waarde'] = df['U_waarde'].round(3)

#U-waardes die boven de 100 afkoppen naar 100 en van negative U-waardes een 0 maken.
def RoundU():
    df['U_waarde'] = df['U_waarde'].mask(df['huid. compr.frequentie  [Hz]']==0, other='')
    df['U_waarde'] = pd.to_numeric(df['U_waarde'])
    df['U_waarde'] = df['U_waarde'].mask(df['U_waarde']>150, other=150)
    df['U_waarde'] = df['U_waarde'].mask(df['U_waarde']<0, other=0)
    
#De definities van de berekende waarden bijelkaar
#De een foutmelding geven als er iets fout is
def calcualteValues():
    try:
        MakeDT()
        MakePa()
        MakeCOP2()
        MakeU()
        RoundU()
        MakeFreezeminutes()
        with out3:
            clear_output()
            print('successfully calculated')
            print('All values:')
            print(list(df))
    except Exception as e:
        with out3:
            clear_output()
            print('error:')
            print(e)

#Knop 4 definieren en weergeven
button4 = widgets.Button(description='Calculate Values')
display(button4)

def on_button4_clicked(b):
    calcualteValues()
    
button4.on_click(on_button4_clicked)

Button(description='Calculate Values', style=ButtonStyle())

In [9]:
out3

Output()

In [10]:
#Knop 3 definieren en weergeven
button3 = widgets.Button(description='Calculate extra Values')
display(button3)
out1 = Output()

#Gemiddelde U berekenen
def meanU():
    print('Mean U: ' + str(round(df['U_waarde'].mean(),3)))

#Afronden naar elke 3 uur
def GroupBy3H():
    global df
    df = df.apply(pd.to_numeric).round(3)
    df = df.groupby(pd.Grouper(freq='3H')).mean()

#COP formule berekenen
def COP_formula():
    global df_sort
    df_sort = df.sort_values('buitentemp. BT1 [°C]')
    df_sort = df_sort.dropna()
    df_sort = df_sort.loc[df['buitentemp. BT1 [°C]'] <= 21]
    z = np.polyfit(df_sort['buitentemp. BT1 [°C]'],df_sort['COP'],1) #Trendlijn vinden
    z = z.round(3)
    print()
    print('Formulas: (outside temp. = x)')
    print("COP formuls: COP(x) = " + "{0}x + {1}".format(*z)) #formule in een begrijpbaar formaat
    print('standaard COP formula: COP(x) = 0.24x + 4.19') #print standaard COP formule

#frequentie formule berekenen
def Freq_formula():
    z = np.polyfit(df_sort['buitentemp. BT1 [°C]'],df_sort['huid. compr.frequentie  [Hz]'],1)
    z = z.round(3)
    print("compressor frequency formula: compressor frequency(x) = " + "{0}x + {1}".format(*z))

#Bron in formule berekenen
def bron_in_formula():
    z = np.polyfit(df_sort['buitentemp. BT1 [°C]'],df_sort['temp. bron in EB100-EP14-BT10 [°C]'],2)
    z = z.round(3)
    print("Brine in formula: Brine in(x) = " + "{0}x^2 + {1}x + {2}".format(*z))
    global bron_f #Global definitie aanmaken om de formule later te gebruiken
    bron_f = z

#Conclusies trekken frequentie formule
def Freq_conclusie():
    global df_sort
    z = np.polyfit(df_sort['buitentemp. BT1 [°C]'],df_sort['huid. compr.frequentie  [Hz]'],1)
    print()
    print('compressor frequency statistics:')
    print('0 compressor frequency at: ' + str(round((z[1] / z[0] *-1),1)) + ' °C')
    print('compressor frequency at 0°C: ' + str(round(z[1],1)) + ' Hz')
    print('Maximum compressor frequency: ' + str(max(df_sort['huid. compr.frequentie  [Hz]'])) + ' Hz')
    print('Maximum compressor frequency at: ' + str(round((max(df_sort['huid. compr.frequentie  [Hz]']) - z[1]) / z[0],1)) + ' °C')
    global freq_f
    freq_f = z

#Conclusies trekken COP formule    
def COP_conclusie():
    z = np.polyfit(df_sort['buitentemp. BT1 [°C]'],df_sort['COP'],1)
    print()
    print('COP statistics:')
    print('1 COP at : ' + str(round(((z[1]-1) / z[0] * -1),1)) + ' °C')
    print('COP at 0°C: ' + str(round(z[1],1)))
    print('COP at 15°C: ' + str(round(z[1] + z[0] * 15,1)))
    global COP_f
    COP_f = z

#Het Totaal COP berekenen
def COP_total():
    powerdeliverd = (1 - 1 / df['COP']) * df['Actueel vermogen'] * 1000
    powerused = (1 / df['COP']) * df['Actueel vermogen'] * 1000
    powerdeliverd = powerdeliverd * 3 * 3600
    powerused = powerused * 3 * 3600
    powerdeliverd = powerdeliverd.sum().round(2)
    powerused = powerused.sum().round(2)
    powerdeliverd = powerdeliverd / 3600000
    powerused = powerused / 3600000

    print('Power deliverd: ' + str(round(powerdeliverd)) + ' kWh')
    print('Power used: ' + str(round(powerused)) + ' kWh')
    print('COP total: ' + str(round(powerdeliverd / powerused)))

#De definities van de formules en de conclusies bij elkaar
#en een foutmelding geven als er iets fout is   
def calcualteExtraValues():
    try:
        meanU()
        GroupBy3H()
        COP_formula()
        Freq_formula()
        bron_in_formula()
        Freq_conclusie()
        COP_conclusie()
        COP_total()
        with out1:
            clear_output()
            print('successfully calculated')
    except Exception as e:
        with out1:
            clear_output()
            print('error:')
            print(e)

def on_button3_clicked(b):
    calcualteExtraValues()
    
button3.on_click(on_button3_clicked)

Button(description='Calculate extra Values', style=ButtonStyle())

In [11]:
out1

Output()

In [12]:
#know 1 defineren
button1 = widgets.Button(description='Show plot lines')
display(button1)

#COP grafiek tekenen
def COP_plot():
    x = np.array(range(-10,20))
    y = COP_f[0]*x + COP_f[1]
    plt.plot(x,y)
    plt.title('COP by outside temperature')
    plt.show()

#Frequentie grafiek tekenen    
def freq_plot():
    x = np.array(range(-10,20))
    y = freq_f[0]*x + freq_f[1]
    plt.plot(x,y)
    plt.title('compressor frequency by outside temperature')
    plt.show()

#Bron in grafiek tekenen
def bron_plot():
    x = np.array(range(-10,20))
    y = bron_f[0]*x**2 + bron_f[1]*x + bron_f[2]
    plt.plot(x,y)
    plt.title('Brine in by outside temperature')
    plt.show()

def ShowPlots():
    COP_plot()
    freq_plot()
    bron_plot()

def on_button1_clicked(b):
    ShowPlots()
    
button1.on_click(on_button1_clicked)

Button(description='Show plot lines', style=ButtonStyle())

In [13]:
#knop 2 defineren
button2 = widgets.Button(description='Show scatter plots')
display(button2)

#spreidingsdiagram voor de COP
def COP_plot_sc():
    plt.scatter(df_sort['buitentemp. BT1 [°C]'], df_sort['COP'])
    plt.title('BT to COP')
    plt.show()

#spreidingsdiagram voor de frequntie
def freq_plot_sc():
    plt.scatter(df_sort['buitentemp. BT1 [°C]'], df_sort['huid. compr.frequentie  [Hz]'])
    plt.title('BT to compressor frequency')
    plt.show()

#spreidingsdiagram voor de bron in
def bron_plot_sc():
    plt.scatter(df_sort['buitentemp. BT1 [°C]'], df_sort['temp. bron in EB100-EP14-BT10 [°C]'])
    plt.title('BT to brine in')
    plt.show()

def Scatterplots():
    COP_plot_sc()
    freq_plot_sc()
    bron_plot_sc()

def on_button2_clicked(b):
    Scatterplots()
    
button2.on_click(on_button2_clicked)

Button(description='Show scatter plots', style=ButtonStyle())

In [14]:
#knop voor dataframe weergeven
button0 = widgets.Button(description='Show dataframe')
display(button0)
out = Output()

#Definitie knop weergeven 
def Showdf():
    with out:
        clear_output()
        display(df)

def on_button0_clicked(b):
    Showdf()
    
button0.on_click(on_button0_clicked)

out

Button(description='Show dataframe', style=ButtonStyle())

Output()

In [15]:
#Knop om het dataframe op te slaan op de schijf
def savefile():
    root = tk.Tk()
    root.lift()
    root.focus_force()
    root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("CSV","*.csv"),("all files","*.*")))
    root.withdraw()
    df.to_csv(root.filename, sep=';')
    print (root.filename)

In [16]:
button6 = widgets.Button(description='Save dataframe')
display(button6)

def on_button6_clicked(b):
    savefile()
    
button6.on_click(on_button6_clicked)

Button(description='Save dataframe', style=ButtonStyle())

In [17]:
#knop voor plotly grafiek
button7 = widgets.Button(description='Make Plotly')
display(button7)

def on_button7_clicked(b):
    cf.go_offline()
    init_notebook_mode(connected=True)
    df.iplot(filename='cufflinks/line-example')

button7.on_click(on_button7_clicked)

Button(description='Make Plotly', style=ButtonStyle())

In [18]:
#plotly maken met de formule van de voorspelde pomp frequentie
def freq_compare():
    df_per = pd.DataFrame()
    df_per['Bt to pump'] = df['buitentemp. BT1 [°C]'] * freq_f[0] + freq_f[1]
    df_per['current compr. frequency'] = df['huid. compr.frequentie  [Hz]']
    df_per['compr. frequency surplus'] = df_per['current compr. frequency'] -  df_per['Bt to pump']
    cf.go_offline()
    init_notebook_mode(connected=True)
    df_per.iplot(filename='cufflinks/line-example')

In [19]:
button8 = widgets.Button(description='freq_compare Plotly')
display(button8)

def on_button8_clicked(b):
    freq_compare()

button8.on_click(on_button8_clicked)

Button(description='freq_compare Plotly', style=ButtonStyle())

In [20]:
#plotly maken met de formule van de voorspelde COP
def COP_compare():
    df_perC = pd.DataFrame()
    df_perC['Bt to COP'] = df['buitentemp. BT1 [°C]'] * 0.24 + 4.19
    df_perC['COP'] = df['COP']
    df_perC['COP surplus'] = df_perC['COP'] -  df_perC['Bt to COP']
    cf.go_offline()
    init_notebook_mode(connected=True)
    df_perC.iplot(filename='cufflinks/line-example')

In [21]:
button10 = widgets.Button(description='COP compare Plotly')
display(button10)

def on_button10_clicked(b):
    COP_compare()

button10.on_click(on_button10_clicked)

Button(description='COP compare Plotly', style=ButtonStyle())

In [22]:
#vriesminuten grafiek maken
button9 = widgets.Button(description='Show freezeminutes')
display(button9)

def on_button9_clicked(b):
    df['Freezeminutes'].plot(use_index = True, kind = 'line', legend = True, sharex=True)
button9.on_click(on_button9_clicked)

Button(description='Show freezeminutes', style=ButtonStyle())